In [ ]:

import pandas as pd
import camelot
import itertools
import PyPDF2 
import numpy as np
from os import path
from glob import glob  
import pdfplumber
import os

def find_ext(dr, ext):
    return glob(path.join(dr,"*.{}".format(ext)))

count = 0
final_df = []

for pth in find_ext("/Users/sid/Documents/MA/SK (11)","pdf"):
    # if pth.endswith('.pdf'):
    pagenolist = []
    final=pd.DataFrame()
    pdfFileObj = open(pth, 'rb')  
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    nopages = pdfReader.numPages
    for pageno in range(0,nopages):
        pageObj = pdfReader.getPage(pageno)  
        if "1. Location Parameters" in pageObj.extractText()[0:22]:
            pagenolist.append(pageno)
    pdfFileObj.close()
    # print('check1')
    for pg in range(0,len(pagenolist)):
        table = camelot.read_pdf(pth, pages=str(pagenolist[pg]+1))
        x_df  = table[0].df
        x = x_df.to_numpy()
        dic = {}
        data = list(itertools.chain.from_iterable(x))
        # print('check2')
        for index in range(0,len(data)):
            if len(data[index]) == 0:
                continue
            if ":" in data[index]:
                ls = data[index].split(":")
                check = ls[0].strip()
                if len(ls[1].strip()) == 0:
                    dic[check] = data[index+1].strip()
                else:
                    dic[check] = ls[1].strip()
        # print('hello')           
        final = pd.DataFrame()
        pdf = pdfplumber.open(pth)
        
        if pg == len(pagenolist)-1:
            endno = nopages
        else:
            endno = pagenolist[pg+1]
        
        for page in range(pagenolist[pg],endno):
            p = pdf.pages[page]
            tab = p.extract_table()
            tab = np.array(tab)
            df1 = pd.DataFrame(tab)
            final = pd.concat([final,df1],axis = 0, ignore_index = True)

        final.columns = final.iloc[2]
        final.drop([0,1,2],axis=0,inplace = True)
        paralist = final["S.NO"].to_list()
        lsP = []
        parameter = "Basic Parameters"
        for row in range(0,len(paralist)):
            if paralist[row].isnumeric():
                lsP.append(parameter)
            else:
                if "Maximum weightage for" in paralist[row]:
                    final.drop(row+3,axis=0,inplace = True)
                else:
                    parameter = paralist[row]
                    final.drop(row+3,axis=0,inplace = True)
        final.drop("S.NO",axis=1,inplace = True)
        final.insert(0,"Parameters",lsP)
        for colna in dic.keys():
            if colna == "Serial No/Total Village":
                continue
            final[colna] = dic[colna] 
            
        def replaceMultiple(mainString, toBeReplaces, newString):
            for elem in toBeReplaces :
                if elem in mainString :
                    mainString = mainString.replace(elem, newString)
            return  mainString
        
        st = replaceMultiple(dic["State"], ["\n", "/", "*","?",":","\",""","<",">","|"] , " ")
        dis = replaceMultiple(dic["District"], ["\n", "/", "*","?",":","\",""","<",">","|"] , " ")
        vil = replaceMultiple(dic["Village"], ["\n", "/", "*","?",":","\",""","<",">","|"] , " ")
        gp = replaceMultiple(dic["Gram Panchayat"], ["\n", "/", "*","?",":","\",""","<",">","|"] , " ")

        pathS = '/Users/sid/Documents/MA_pdf/'+st
        isdirS = os.path.isdir(pathS) 

        pathD = '/Users/sid/Documents/MA_pdf/'+st+"/"+dis
        isdirD = os.path.isdir(pathD)
        
        pathV = pathD+"/" + gp +"_"+ vil +".csv"
        isfileV = os.path.isfile(pathV)
        
        pathdupli = pathS+"/duplicates"
        isdirDupli = os.path.isdir(pathdupli)

        if isdirS:
            if isdirD:
                
                if isfileV:
                    print("duplicate")
                    print(pth)
                    
                    if isdirDupli:
                        final.to_csv(pathS+"/duplicates/"+gp+"_"+ vil +".csv",index = 0)
                        count+=1
                    else:
                        directory = "duplicates"
                        parent_dir = pathS
                        path = os.path.join(parent_dir, directory)
                        os.mkdir(path) 
                        final.to_csv(pathS+"/duplicates/"+gp+"_"+ vil +".csv",index = 0)
                        count+=1
                else:
                    final.to_csv(pathV,index = 0)
                    count+=1
            else:
                directory = dis
                parent_dir = pathS
                path = os.path.join(parent_dir, directory)
                os.mkdir(path) 
                print("Directory '% s' created" % directory)  
                final.to_csv(pathV,index = 0)
                count+=1

        else:
            directory = st
            parent_dir = '/Users/sid/Documents/MA_pdf/'
            path = os.path.join(parent_dir, directory)
            os.mkdir(path) 
            directory = dis
            parent_dir = pathS
            path = os.path.join(parent_dir, directory)
            os.mkdir(path) 
            print("Directory '% s' created" % directory)  
            final.to_csv(pathV,index = 0)
            count+=1
    final_df.append(final)
print(count)
# final